# このCodeの目的
- 3次元画像の画素値の分布確認
- 画像ごとに、どんな確率分布に従うか分析

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import os, shutil
from tqdm import tqdm

In [ ]:
# case × day の組み合わせ（=3次元画像数）を取得
df_train = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
df_train['case'] = df_train['id'].apply(lambda x: int(x.split('_')[0].replace('case', '')))
df_train['day'] = df_train['id'].apply(lambda x: int(x.split('_')[1].replace('day', '')))
df_train['slice'] = df_train['id'].apply(lambda x: x.split('_')[3])

# path_partial：idから画像データの path 情報を部分的に作成（4つの数字が分からない）
TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train'
all_train_images = glob(os.path.join(TRAIN_DIR, '**', '*.png'), recursive=True)
x = all_train_images[0].rsplit('/', 4)[0] ## ../input/uw-madison-gi-tract-image-segmentation/train

path_partial_list = []
for i in range(0, df_train.shape[0]):
    path_partial_list.append(os.path.join(x,
                          'case'+str(df_train['case'].values[i]),
                          'case'+str(df_train['case'].values[i])+'_'+ 'day'+str(df_train['day'].values[i]),
                          'scans',
                          'slice_'+str(df_train['slice'].values[i])))
df_train["path_partial"] = path_partial_list

# inputフォルダから、直接正しいpath情報を取得し、path_partialと対応付け
path_partial_list = []
for i in range(0, len(all_train_images)):
    path_partial_list.append(str(all_train_images[i].rsplit('_',4)[0]))
    
tmp_df = pd.DataFrame()
tmp_df['path_partial'] = path_partial_list
tmp_df['path'] = all_train_images

# path 情報列追加
df_train = df_train.merge(tmp_df, on='path_partial').drop(columns=['path_partial'])

df_train['width'] = df_train['path'].apply(lambda x: int(x[:-4].rsplit('_',4)[1]))
df_train['height'] = df_train['path'].apply(lambda x: int(x[:-4].rsplit('_',4)[2]))
df_train['width_space'] = df_train['path'].apply(lambda x: float(x[:-4].rsplit('_',4)[3]))
df_train['height_space'] = df_train['path'].apply(lambda x: float(x[:-4].rsplit('_',4)[4]))

label = ['case','day','slice','width','height','width_space','height_space']
df_train_3d = df_train.sort_values('slice').loc[:,label].drop_duplicates(subset=['case','day']).sort_values(['case', 'day']).reset_index().drop('index', axis=1)
df_train = df_train.drop_duplicates(subset=['case','day','slice']).reset_index().drop('index', axis=1)

In [ ]:
def make_2d_img_array(CASE, DAY, SLICE):
    # train画像のpath取得
    TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train/case'+str(CASE)+'/case'+str(CASE)+'_day'+str(DAY)+'/scans/'
    train_image = glob(os.path.join(TRAIN_DIR, '**', 'slice_' + SLICE + '*.png'), recursive=True)

    # 2次元画像を格納
    im = np.asarray(Image.open(train_image[0]))

    return im

def make_3d_img_array(CASE, DAY):
    # train画像のpath取得
    TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train/case'+str(CASE)+'/case'+str(CASE)+'_day'+str(DAY)+'/scans/'
    train_images = glob(os.path.join(TRAIN_DIR, '**', '*.png'), recursive=True)
    train_images = sorted(train_images)

    # 画像のサイズ取得
    im = np.asarray(Image.open(train_images[0]))
    height = len(im)
    width = len(im[0])

    # 2次元画像を3次元配列に格納
    im_3d = np.empty((0, height, width))

    for path in train_images:
        im = np.asarray(Image.open(path))
        im = np.reshape(im, (1, height, width))
        im_3d = np.append(im_3d, im, axis = 0)

    #print('size:{}'.format(im_3d.shape))
    return im_3d

# 1. 3次元画像の輝度値の基礎統計量
各3次元画像の 最大値、平均値、中央値、最小値を確認

In [ ]:
df_train_3d['max_bright']    = 0
df_train_3d['mean_bright']   = 0
df_train_3d['median_bright'] = 0
df_train_3d['min_bright']    = 0

# 全ての3次元画像ループ　2次元画像38,434枚読み込み
for i in tqdm(df_train_3d.index):
    # 3次元配列作成
    CASE = df_train_3d.loc[i,'case']
    DAY  = df_train_3d.loc[i,'day']
    im_3d = make_3d_img_array(CASE, DAY)
    
    # 3次元画像の基礎統計量
    df_train_3d.loc[i,'max_bright']    = np.max(im_3d)
    df_train_3d.loc[i,'mean_bright']   = np.mean(im_3d)
    df_train_3d.loc[i,'median_bright'] = np.median(im_3d)
    df_train_3d.loc[i,'min_bright']    = np.min(im_3d)

In [ ]:
df_train_3d.describe()

学習データは、全ての case day で輝度値が255を超える<br>
最も低い3次元画像でも、輝度値が659まである。<br>
学習時は、この最大輝度値で割ることで、0-1の値に正規化する必要があるかも。

In [ ]:
labels = ['max_bright','mean_bright','median_bright','min_bright']

plt.figure(figsize=(20,3))
i=1
for label in labels:
    plt.subplot(1,4,i) 
    plt.hist(df_train_3d.loc[:,label],bins=50)
    plt.title(label)
    i+=1

plt.show()

最大値のヒストグラムは、完全に２つに分かれている。<br>
case ごとに、MRI撮影条件が異なるのかもしれない。

# 2. 3次元画像ごとの輝度値が従う確率分布
MRI画像は、SNRが2より大きいピクセルは正規分布に、2以下のピクセルはRice分布に、信号がない箇所はRayleigh分布に従うことが知られている。<br>
それぞれのピクセルがどの確率分布に従うか検証し、ノイズの影響を推定する。
参考：https://pubmed.ncbi.nlm.nih.gov/8598820/

In [ ]:
from fitter import Fitter
import random

In [ ]:
# 確率分布の表示
def show_fitter(im):
    f = Fitter(im, distributions=['norm', 'rice', 'rayleigh'])
    f.fit()
    f.summary()
    plt.show()

def show_img(im):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    img = ax.imshow(np.asarray(im),cmap=plt.cm.jet)
    cbar = fig.colorbar(img, ax=ax, aspect=50, pad=0.08, shrink=0.9, orientation='vertical')
    plt.show()

In [ ]:
# 表示する3次元画像をランダムに10選択
idxs = random.sample(range(len(df_train_3d)), k=10)

for idx in idxs:
    CASE = df_train_3d.loc[idx,'case']
    DAY  = df_train_3d.loc[idx,'day']
    im_3d = make_3d_img_array(CASE, DAY)
    
    # 1次元配列に変換して、10000件サンプリング
    im_3d = random.sample(np.ravel(im_3d).tolist(), 10000)
    print('case {}, day {}'.format(CASE,DAY))
    show_fitter(im_3d)

3次元画像では、どの分布にも従っているように見えなかった。<br>
そのため、配布された形式の、スライスごとの2次元画像でも確認する。

In [ ]:
# 表示する2次元画像をランダムに30選択
idxs = random.sample(range(len(df_train)), k=10)

for idx in idxs:
    CASE  = df_train.loc[idx,'case']
    DAY   = df_train.loc[idx,'day']
    SLICE = df_train.loc[idx,'slice']
    im_2d = make_2d_img_array(CASE, DAY, SLICE)
    
    # 1次元配列に変換して、10000件サンプリング
    im_2d = random.sample(np.ravel(im_2d).tolist(), 10000)
    print('case {}, day {}, slice {}'.format(CASE,DAY,SLICE))
    show_fitter(im_2d)

輝度値の低いところを除けば、山なりの分布になっているものもあるが、大多数は山が確認できなかったり、2つの山に分かれているものが多い。

In [ ]:
print('山が確認できる例')
CASE  = 141
DAY   = 27
SLICE = '0045'
im_2d = make_2d_img_array(CASE, DAY, SLICE)

print('case {}, day {}, slice {}'.format(CASE,DAY,SLICE))

# 確率分布の表示
im = random.sample(np.ravel(im_2d).tolist(), 50000)
show_fitter(im)

# 画像表示
show_img(im_2d)

In [ ]:
print('山が確認できない例')
CASE  = 124
DAY   = 19
SLICE = '0122'
im_2d = make_2d_img_array(CASE, DAY, SLICE)

print('case {}, day {}, slice {}'.format(CASE,DAY,SLICE))

# 確率分布の表示
im = random.sample(np.ravel(im_2d).tolist(), 50000)
show_fitter(im)

# 画像表示
show_img(im_2d)

In [ ]:
print('山が2つに分かれる例')
CASE  = 55
DAY   = 0
SLICE = '0110'
im_2d = make_2d_img_array(CASE, DAY, SLICE)

print('case {}, day {}, slice {}'.format(CASE,DAY,SLICE))

# 確率分布の表示
im = random.sample(np.ravel(im_2d).tolist(), 50000)
show_fitter(im)

# 画像表示
show_img(im_2d)

きれいな確率分布は確認できなかった。以下のようなことを更に試せると、何か分かるかもしれない。
- 正規化した上で、外れ値を除外して確率分布を確認する。
- フーリエ変換により、周波数領域でノイズの特徴がないか確認する。